# Construction d'un système de recommandation

Nous avons décidé d'orienter notre projet sur la recommendation de films.
En effet durant ce confinement, nous avons eu le temps de visionner beaucoup de films,
mais nous nous sommes rendus compte que nous passions quasiment autant de temps
à choisir le film qu'à le regarder. D'où la nécessité de créer un système de re-
commendations afin d'optimiser notre temps de visionnage.
Nous avons chercher une base de données assez exploitable afin de mener à bien
notre projet. Nous nous sommes basés sur la base de données de 'The Movies Dataset'.

# Différents systèmes de recommandation

- [x] popularity based = moyenne simple
- [x] memory-based (user- et item- based)
- [x] hybride : popularity/collabo
- [x] clustering
- [ ] hybride : cluster/collabo
- [ ] model-based (matrix factorisation, optimisation avec descente de gradient)
    - [x] descente de gradient
    - [ ] cross-validation pour tuner les hyperparamètres
- [ ] hybride : cluster/model
- [ ] user-centered linear approach = descente de gradient (même pb d'opti que model-based, mais on donne les infos des films)


In [1]:
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import re
from time import time
from ast import literal_eval

## Fetching and cleaning data

Nous utilisons deux tables de données. L'une, *movies_metadata.csv*, contient une liste de films et des informations relativesau genre, date de sortie etc. 

### Informations sur les films

In [14]:
movies = pd.read_csv("movies_metadata.csv")
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [15]:
movies.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [16]:
def filter_correct_id(word):
    if re.fullmatch(r'[0-9]+', word):
        return word
    return "wrong_id"

In [17]:
# don't re-run
movies = movies[~movies.id.duplicated()]
movies.id = movies.id.apply(filter_correct_id)
movies = movies[movies.id != "wrong_id"]
movies.id = movies.id.astype('int64')

### Avis des utilisateurs

In [18]:
ratings = pd.read_csv("ratings_small.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [19]:
# ne pas re-run !
ratings = ratings.drop(columns=['timestamp'])
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [20]:
ratings[(ratings['userId'] == 1) & (ratings['movieId'] == 31)]

,userId,movieId,rating
0,1,31,2.5


In [21]:
print(min(ratings.rating), max(ratings.rating))
ratings.describe()
ratings.dtypes

0.5 5.0


userId       int64
movieId      int64
rating     float64
dtype: object

Dans tout le notebook, on considère l'existence d'une variable globale `dfr` contenant la dataframe des notes et `dfm` contenant la dataframe des films. Cela nous permet d'abord tester notre code sur des petits échantillons et avant de les faire tourner sur la totalité des donneés, sans avoir à modifier le code. De même,

## Top films par genres

### Selection des genres

In [22]:
dfr = ratings.copy()
dfm = movies.copy().rename(columns={'id' : 'movieId'})
dfm = dfm.merge(dfr, how='inner')
dfm = dfm.loc[:, ['movieId', 'genres', 'title']]
dfm.head()
# dfm.sort_values(by='movieId').head(100)
# dfr.head(20)

,movieId,genres,title
0,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat
1,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat
2,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat
3,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat
4,949,"[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...",Heat


In [23]:
def simplify_genre(l):
    if len(l) <= 0 :
        return []
    if isinstance(l[0], dict):
        return [d['name'] for d in l]
    return l

dfm['genres'] = dfm['genres'].apply(lambda x: literal_eval(x) if isinstance(x, str) else x).apply(simplify_genre)
dfm.head()

,movieId,genres,title
0,949,"[Action, Crime, Drama, Thriller]",Heat
1,949,"[Action, Crime, Drama, Thriller]",Heat
2,949,"[Action, Crime, Drama, Thriller]",Heat
3,949,"[Action, Crime, Drama, Thriller]",Heat
4,949,"[Action, Crime, Drama, Thriller]",Heat


### Normaliser les notes sur toutes les données

In [24]:
%timeit dfm['mean_rating'] = ratings.groupby('movieId').mean().loc[:, ['rating']]

14.8 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [25]:
dfm.head()

,movieId,genres,title,mean_rating
0,949,"[Action, Crime, Drama, Thriller]",Heat,NaN
1,949,"[Action, Crime, Drama, Thriller]",Heat,3.872470
2,949,"[Action, Crime, Drama, Thriller]",Heat,3.401869
3,949,"[Action, Crime, Drama, Thriller]",Heat,3.161017
4,949,"[Action, Crime, Drama, Thriller]",Heat,2.384615


In [26]:
dfm = dfm.sort_values(by='mean_rating', ascending=False)

In [27]:
dfm.head()

,movieId,genres,title,mean_rating
702,1945,"[Drama, Thriller]",Nell,5.0
4711,1092,"[Thriller, Mystery]",The Third Man,5.0
32515,920,"[Animation, Adventure, Comedy, Family]",Cars,5.0
3202,3083,"[Comedy, Drama]",Mr. Smith Goes to Washington,5.0
32460,588,"[Horror, Mystery]",Silent Hill,5.0


# <span style="color:green">copy de dfm + join pour que mean_rating soit respecté + ou ne pas faire de repetition et utiliser deux tables ? </span>

In [33]:
# 1 genre par ligne
serie = dfm.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
serie.name = 'genre'
dfm_simpl = dfm.copy().drop('genre', axis=1).join(serie)
dfm_simpl.head()

,movieId,title,mean_rating,genres
0,949,Heat,NaN,Action
0,949,Heat,NaN,Crime
0,949,Heat,NaN,Drama
0,949,Heat,NaN,Thriller
1,949,Heat,3.87247,Action


In [34]:
def top(genres):
    # selection des films par genre
    df_genres = dfm_simpl.loc[dfm_simpl['genre'].isin(genres)]
    df_genres = df_genres.sort_values(by='mean_rating', ascending=False)
    return df_genres

In [35]:
top(['Drama', 'Comedy']).head()

,movieId,title,mean_rating,genres
26094,2817,Same Old Song,5.0,Comedy
32515,920,Cars,5.0,Comedy
5301,1554,Down by Law,5.0,Comedy
5301,1554,Down by Law,5.0,Drama
3575,2033,Infinity,5.0,Drama


In [36]:
def union_list(lst1, lst2):
    return list(set(lst1) | set(lst2)) 

def pref_genres(uid):
    '''
    Retourne les genres des 5 films préférés de l'user uid
    '''
    rats = dfr.loc[dfr['userId'] == uid, :].sort_values(by='rating')
    pref = rats.head(5)['movieId'].values if rats.shape[0] > 5 else rats['movieId'].values
    genres = []
    for g in dfm.loc[dfm['movieId'].isin(pref)].genres :
        genres = union_list(genres, g)
    return genres

In [37]:
def top_reco(uid, k):
    '''
    Retourne les k films les plus populaires appartenant aux genres préféres de l'user uid
    '''
    chart = top(pref_genres(uid))
    chart = chart.loc[~chart['movieId'].isin(dfr.loc[dfr['userId'] == uid, :].movieId.unique()) ]
    return chart.head(k) if chart.shape[0] > k else chart

In [38]:
user = 4
k = 15

print(pref_genres(user))
top_reco(user, k)

['Comedy', 'Action', 'Science Fiction', 'Romance', 'Adventure', 'Drama', 'Thriller', 'Family']


,movieId,title,mean_rating,genres
3021,223,Rebecca,5.0,Drama
1455,8831,Timecop,5.0,Action
4567,595,To Kill a Mockingbird,5.0,Drama
1428,1689,Little Buddha,5.0,Drama
4565,595,To Kill a Mockingbird,5.0,Drama
26422,5902,A Bridge Too Far,5.0,Drama
4522,595,To Kill a Mockingbird,5.0,Drama
26501,316,Grill Point,5.0,Comedy
26501,316,Grill Point,5.0,Drama
4466,595,To Kill a Mockingbird,5.0,Drama


# <span style="color:red"> À blablater : méthode pas personalisée, privilégie les plus populaires et ne permet pas d'évaluer de manière quantitative la pertinence (pas de note)
</span>

## Collaborative filtering : user- et item- based

Pour prédire la note d'un couple (*user*, *movie*) on peut regarder quelle note les utilisateurs similaires à *user* ont donné à ce film et faire une moyenne de leurs notes. On peut également regarder quelle note *user* a donné à des films similaires à *movie*. La première approche est centré sur les utilisateurs, *user-based*, tandis que la deuxième est centrée sur les films, *item-based*. Neánsmoins les deux approches suivent la même logique et nous allons implémenter des fonctions qui s'adaptent en fonction de l'approche choisie. Dans un système *user-based*, nous allons appeler **peers** les **users** et **others** les **items**. Dans un système *item-based* c'est l'inverse.



Deux utilisateurs sont considérés comme similaires s'ils ont les mêmes préférences de films. Il semble en effet plus pertinent de demander à un utilisateurs aux goûts similaires à *user* de lui conseiller un film. Pour comparer deux utilisateurs il faudra donc regarder les notes qu'ils ont donné aux mêmes films. De manière analogue, deux films sont similaires s'ils sont appréciés par les mêmes utilisateurs. Il faudra donc regarder les notes données par les mêmes utilisateurs pour comparer deux films. Cette notion de similitude sera calculée par un taux de corrélation.

Nous allons considérer une variable globale `cm_user` et `cm_movie` contenant la matrice de correlation entre utilisateurs et films. 

### Normalisation des notes

Nous n'avons besoin pour ce système que des notes données par les utilisateurs. Puisque la moyenne des notes données varie d'un utilisateur à un autre et d'un film à un autre, nous allons translater les notes afin que la moyenne des notes se trouve à 0. En *user-based*, on considère la moyenne par utilisateur, tandis qu'en *item-based* on s'interèsse à la moyenne par film. Par abus de langage nous appelons ces nouvelles notes les notes *normalisées*. 

In [105]:
# test si plus efficace 
# _ mean une dataframe, et donc récuper la note moyenne avec un loc
# _ mean un numpy matrice, avec une liste donnant l'équivalent entre id et indice

dfr = ratings.copy().loc[(ratings['userId'] <= 50) & (ratings['movieId'] <= 500)]
base ='user'
ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
print(dfr.shape)
dfr.head()

(1165, 3)


,userId,movieId,rating
0,1,31,2.5
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0


In [132]:
%%timeit
mean1 = dfr.groupby(ptype).mean().loc[:, ['rating']]
new_col = 'rating_norm_'+base
dfr[new_col] = dfr[[ptype, 'rating']].apply(lambda row : 
                                            row['rating'] -  mean1.loc[int(row[ptype])]['rating'], axis=1)

315 ms ± 24.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [133]:
%%timeit
mean2 = dfr.groupby(ptype).mean()['rating']
new_col = 'rating_norm_'+base
dfr[new_col] = dfr[[ptype, 'rating']].apply(lambda row : 
                                            row['rating'] -  mean2.loc[int(row[ptype])], axis=1)

109 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [136]:
def normalize(base='user'):
    '''
    Ajoute une colonne dans la dataframe df contenant les notes normalisées des utilisateurs
    Retourne la Série donnant la moyenne des notes par peer
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    mean = dfr.groupby(ptype).mean()['rating']
    new_col = 'rating_norm_'+base
    dfr[new_col] = dfr[[ptype, 'rating']].apply(lambda row : 
                                            row['rating'] -  mean.loc[int(row[ptype])], axis=1)
    return mean

### Calculer la matrice de corrélation


Dans un système *user-based*, on note $I_u$ l'ensemble des items renseignés pour l'utilisateur $u$ et $U_k$ l'ensemble des utilisateurs qui ont notés le film $k$. On note $I_{uv} = I_u \cap I_v$. Pour le *item-based* on utilisera les mêmes notations en intervertissant user et item. On notera également $S_{ui}$ la note normalisée de l'item *i* donnée par l'utilisateur *u*. 


Pour déterminer si deux utilisateurs se ressemblent en termes de goûts, nous utilisons un taux de corrélation sur les avis données. Nous allons comparer quatres taux de corrélations différents. Le premier ```cor()``` calcule le taux de corrélation classique donné par la formule :
$$
cor(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk}}{\sqrt{\sum_{k \in I_{uv}} s_{uk}^2}\sqrt{\sum_{k \in I_{uv}} s_{vk}^2}}
$$

Le taux de corrélation ajusté ```cor_adj()``` permet de ne pas donner trop d'importance aux films populaires que beaucoup de personnes ont vu.
$$
cor\_adj(u, v) = \frac{\sum_{k \in I_{uv}} s_{uk} s_{vk} / U_k}{\sqrt{\sum_{k \in I_{uv}} \frac{s_{uk}^2}{|U_k|}}\sqrt{\sum_{k \in I_{uv}} \frac{s_{vk}^2}{|U_k|}}}
$$

Le taux de correlation calculé par ```cor_dis()``` permet de ne pas donner une correlation trop élevée si les deux utilisateurs n'ont pas donné assez d'avis sur des films en commun. 
$$
cor\_dis(u, v) = cor(u, v) * \frac{min(|I_{uv}|, \beta)}{\beta}
$$

Enfin la fonction ```cor_dis_adj()``` fait un mélange des deux dernières amélioration : il filtre les films trop populaire et n'apporte de l'importance seulement si deux personnes ont données leur avis sur un certain nombre de films.

$$
cor\_dis(u, v) = cor\_adj(u, v) * \frac{min(|I_{uv}|, \beta)}{\beta}
$$

# <span style="color:red">Big question : on donne quoi comme correlation si su et/ou sv est nul ? J'ai mis 0 par défaut mais bon ...
</span>

In [199]:
def tx_cor(u, v, base):
    '''
    :param: u, v - les id des peers (user ou movie) à comparer
            base - un indicateur dy type de recommandation utilisé : 'user' ou 'movie'
    :return: le taux de corrélation classique entre u et v.
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    
    Iu = dfr.loc[dfr[ptype] == u, [otype]]
    Iv = dfr.loc[dfr[ptype] == v, [otype]]
    Iuv = Iu.join(Iv.set_index(otype), on=otype, how='inner')
    if not Iuv.shape[0] : # l'intersection est vide
        return float('nan')

    su = dfr.loc[(dfr[ptype] == u) & (dfr[otype].isin(Iuv[otype]))]
    sv = dfr.loc[(dfr[ptype] == v) & (dfr[otype].isin(Iuv[otype]))]
    su = su['rating_norm_'+base].to_numpy()
    sv = sv['rating_norm_'+base].to_numpy()
    
    up = np.dot(su, sv)
    down = math.sqrt(np.dot(su, su) * np.dot(sv, sv))

# default value to change
    if up == 0 or down == 0:
        return 0
    return up / down

# <span style="color:red">À adapter encore</span>

In [41]:
def cor_adj(u, v, df, Iuv):
    nb_rat = df.loc[:, ['movieId', 'rating']].groupby(['movieId']).count()
    
    sum_up = 0
    sum_down_u = 0
    sum_down_v = 0
    for movie in Iuv.movieId.unique() :
        suk = df.loc[(df['userId'] == u) & (df['movieId'] == movie), ['rating_norm_'+base]]
        svk = df.loc[(df['userId'] == v) & (df['movieId'] == movie), ['rating_norm_'+base]]
        suk, svk = float(suk), float(svk)
        
        sum_up += suk * svk / nb_rat.at[movie, 'rating']
        sum_down_u += suk**2 /  nb_rat.at[movie, 'rating']
        sum_down_v += svk**2 /  nb_rat.at[movie, 'rating']
    return sum_up / math.sqrt(sum_down_u * sum_down_v)

In [42]:
def cor_dis(u, v, df, Iuv):
    beta = 20
    correlation = cor(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

In [43]:
def cor_dis_adj(u, v, df, Iuv):
    beta = 20
    correlation = cor_adj(u, v, df, Iuv)
    return correlation * min(len(Iuv), beta)/beta

# <span style="color:red">Big question 2 : un gars ([ici](https://www.ethanrosenthal.com/2015/11/02/intro-to-collaborative-filtering/)) fait cette fonction pour calculer la matrice plus rapidement </span>


In [44]:
def fast_similarity(ratings, kind='user', epsilon=1e-9):
    # epsilon -> small number for handling dived-by-zero errors
    if kind == 'user':
        sim = ratings.dot(ratings.T) + epsilon
    elif kind == 'item':
        sim = ratings.T.dot(ratings) + epsilon
    norms = np.array([np.sqrt(np.diagonal(sim))])
    return (sim / norms / norms.T)

Nous construisons maintenant la matrice de correlation. Puisqu'une telle matrice est symétrique, nous avons préféré utiliser une dataframe à deux entrées et ainsi ne stocker la corrélation pour un couple qu'une seule fois. En procédant comme tel, l'ordre dans lequel on désigne un couple peer-peer sera important. Pour faciliter l'accès, nous trions d'abord la datframe `df` pour que les peers soient pris dans l'ordre croissant des id. Ainsi les doubles indices de la dataframe construite auront tous la propriété que le premier indice est strictement inférieur au deuxième. Lors de l'accès à la correlation entre deux peers *u et v* il suffira de les ranger dans le bon ordre.

La fonction de corrélation à utiliser peut être précisée en argument et par défaut la fonction choisie est la corrélation classique.

Nous utilisons également le module logging pour suivre le déroulement du calcul. Celui-ci peut être très long en fonction de la taille des données et en rappelant tous les 10 peers qu

Comme le calcul de la matrice peut-être très long, afin d'voir un suivi du déroulement du calcul, on utilise le module logging. Cette fonctionalité est désactivée par défaut. 

# <span style="color:green">Comparer temps de construction d'une matrice de corrélation vs dataframe de corrélation. Dans construction de matrice, peut-être checker si déjà calculer et recuperer dans cellule déjà entrée ? Ou est ce plus long que de juste recalculer le taux de corrélation ? ==> timer le calcul d'une correlation </span>

In [280]:
dfr = ratings.copy().loc[(ratings['userId'] <= 50) & (ratings['movieId'] <= 500)]
print(dfr.shape)
base ='user'
user = 2
movie = 4

mean_user = normalize(base)

(1165, 3)


In [281]:
%timeit cor(u, v, base)
%timeit 3 + 4

16.7 ms ± 3.09 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
32.2 ns ± 2.49 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [282]:
%timeit peers = dfr.userId.unique() if base == 'user' else dfr.movieId.unique()

304 µs ± 34.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [283]:
%timeit peers = dfr[base+'Id'].unique()

301 µs ± 33.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [287]:
%timeit cm_user = slow_cor_matrix(base='user')

16.8 s ± 2.27 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [294]:
%timeit cm_user, user_rank = cor_matrix(base='user')

15.8 s ± 6.64 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [296]:
%%timeit
global cm_user
global user_rank
cm_user = slow_cor_matrix(base='user')
a = slow_get_cor(u, v, base)

25.3 s ± 7.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [295]:
%%timeit
global cm_user
global user_rank
cm_user, user_rank = cor_matrix(base='user')
a = get_cor(u, v, base)

16.8 s ± 2.07 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [276]:
def slow_cor_matrix(cor_fct=tx_cor, base='user', verbose=False):
    '''
    Retourne la dataframe des taux de corrélations entre peers
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    dfr.sort_values(by=ptype, inplace=True)
    peers = dfr[base+'Id'].unique()
    nb_peers = len(peers)
    
    if verbose:
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        logging.info('nb of peers: {}'.format(nb_peers))
    
    correlation = []
    couples = []
    for i in range(nb_peers):
        u = peers[i]
        if verbose and not i % 10 : 
            logging.info('peer nb: {} (id {})'.format(i, u))
        for j in range(i + 1, nb_peers):
            v = peers[j]
            tx_cor = cor_fct(u, v, base)
            if not np.isnan(tx_cor):
                couples.append((u, v))
                correlation.append(tx_cor)
    index = pd.MultiIndex.from_tuples(couples, names=['u', 'v'])
    cor = pd.DataFrame(correlation, index=index, columns=['correlation'])
    return cor

In [277]:
def slow_get_cor(u, v, base='user'):
    '''
    Retourne le taux de correlation entre u et v stocké
    '''
    cm = cm_user if base == 'user' else cm_movie
    if u > v :
        u, v = v, u
    index = list(cm.index.values)
    if (u, v) in index :
#         print('here')
        return float(cm.loc[(u, v)])
    return float('nan')

In [292]:
def cor_matrix(cor_fct=tx_cor, base='user', verbose=False):
    '''
    Retourne la matrice de corrélation entre peers
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    dfr.sort_values(by=ptype, inplace=True)
    peers = dfr[base+'Id'].unique()
    nb_peers = len(peers)
    
    if verbose:
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        logging.info('nb of peers: {}'.format(nb_peers))
    
    cor = np.empty((nb_peers,nb_peers))
    cor[:] = np.nan

    for i in range(nb_peers):
        u = peers[i]
        if verbose and not i % 10 : 
            logging.info('peer nb: {} (id {})'.format(i, u))
        for j in range(i + 1, nb_peers):
            v = peers[j]
            tx_cor = cor_fct(u, v, base)
            if not np.isnan(tx_cor):
                cor[i, j] = tx_cor
                cor[j, i] = cor[i, j]
    return cor, peers

In [293]:
def get_cor(u, v, base='user'):
    '''
    Retourne le taux de correlation entre u et v
    '''
    cm = cm_user if base == 'user' else cm_movie
    id_index = user_rank if base == 'user' else movie_rank
    idu, idv = np.argwhere(id_index == u)[0, 0], np.argwhere(id_index == v)[0, 0]
    return cm[u, v]

### Prédiction

Pour prédire la note donnée par un utilisateur à un film, nous allons faire un moyenne des notes données pour les k peer les plus proches. Dans une approche user-based, on regarde donc les k plus proches utilisateurs, dans une approche item-based, les k films les plus proches. Les plus proches sont ceux qui ont une corrélation la plus élevée. On appelle **p** le peer et **o** l'élément other.

La moyenne effectuée est pondérée par les coefficients de corrélations. On ajoute également la moyenne des notes de **p** pour retrouver une note non normalisée.


$$
\hat{\sigma}_{um} = \mu_u + \frac{\sum_{v \in P_u(m)} s_{vm} \cdot cor(u, v)}{\sum_{v \in P_u(m)} |cor(u,v)|}
$$

# <span style="color:red">Pourquoi utiliser un parsing fait maison et pas knn non parametré comme tout le monde sur internet ?</span>

In [297]:
def peer_group(p, o, base='user', k=4):
    '''
    Retourne les k peers les plus proches de p
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')
    
    # in user-based, get users that rated the movie o
    peers = dfr.loc[dfr[otype] == o, [ptype, 'rating_norm_'+base]]
    peers = peers[base+'Id'].unique()
    
    top = [(float('-inf'), p)] * k
    for v in peers:
        taux = get_cor(p, v, base)
        if taux > top[-1][0] :
            top += [(taux, v)]
            top.sort(reverse=True)
            top = top[:-1]
    return [t[1] for t in top]

# <span style="color:red">Que faire si sum_dow == 0 ? + prédire une note possible ?</span>

In [298]:
def predict_collabo(p, o, base='user', k=4):
    '''
    Retourne la prédiction de la note du couple (p, o) utilisant un peer-groupe de taille k
    '''
    ptype, otype = ('userId', 'movieId') if base == 'user' else ('movieId', 'userId')

    mu = mean_user[p] if base == 'user' else mean_movie[p]
    peers = peer_group(p, o, base, k)
    
    sum_up, sum_down = 0, 0
    for friend in peers:
        cor = get_cor(p, friend, base)
        if not math.isnan(cor):
            sfo = dfr.loc[(dfr[ptype] == friend) & (dfr[otype] == o), 'rating_norm_'+base]
            sfo = 0 if len(sfo) <= 0 else float(sfo)
            sum_up += sfo * cor
            sum_down += abs(cor)

    sum_down = 1 if sum_down == 0 else sum_down
    pred = mu + sum_up / sum_down
    
    # note prédite à une précision de 0.5
    pred = round(2 * pred) / 2
    # note prédite se trouve entre 0 et 5
    pred = 5 if pred > 5 else pred
    pred = 0 if pred < 0 else pred

    return pred

### Test des fonctions de manière individuelle

In [93]:
dfr = ratings.copy().loc[(ratings['userId'] <= 30) & (ratings['movieId'] <= 500)]
print(dfr.shape)
dfr.head()

(783, 3)


,userId,movieId,rating
0,1,31,2.5
20,2,10,4.0
21,2,17,5.0
22,2,39,5.0
23,2,47,4.0


In [71]:
%timeit normalize('user')

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


908 ms ± 94.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
dfr.head()

,userId,movieId,rating,rating_norm_user,rating_norm_movie
0,1,31,2.5,0.000000,-0.678571
75,2,500,4.0,0.642857,0.532680
74,2,497,3.0,-0.357143,-0.983333
73,2,485,3.0,-0.357143,0.113208
72,2,480,4.0,0.642857,0.293796


In [73]:
dfr.shape

(783, 5)

In [94]:
normalize('movie')
normalize('user')
dfr.head()

,userId,movieId,rating,rating_norm_movie,rating_norm_user
0,1,31,2.5,-0.250000,0.000000
20,2,10,4.0,0.666667,0.642857
21,2,17,5.0,1.000000,1.642857
22,2,39,5.0,1.375000,1.642857
23,2,47,4.0,-0.166667,0.642857


In [95]:
cm_user = slow_cor_matrix(base='user', verbose=True)

INFO:root:nb of peers: 28
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 22)


In [96]:
cm_user.head()

correlation
u v             
1 7     0.000000
2 3     0.334284
  4     0.113216
  5    -0.142048
  7     0.241421

In [79]:
%timeit cm_user = cor_matrix(base='user', verbose=True)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 28
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 22)
/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 28
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 22)
/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

11.7 s ± 95.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [352]:
cm_user = cor_matrix(base='user', verbose=True)
cm_user.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 28
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 22)


correlation
u v             
1 7     0.000000
2 3     0.818420
  4     0.327408
  5     0.255686
  7    -0.073746

In [80]:
%timeit cm_movie = slow_cor_matrix(base='movie', verbose=True)

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 260
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 24)
INFO:root:peer nb: 30 (id 40)
INFO:root:peer nb: 40 (id 60)
INFO:root:peer nb: 50 (id 79)
INFO:root:peer nb: 60 (id 100)
INFO:root:peer nb: 70 (id 123)
INFO:root:peer nb: 80 (id 152)
INFO:root:peer nb: 90 (id 164)
INFO:root:peer nb: 100 (id 177)
INFO:root:peer nb: 110 (id 202)
INFO:root:peer nb: 120 (id 223)
INFO:root:peer nb: 130 (id 235)
INFO:root:peer nb: 140 (id 260)
INFO:root:peer nb: 150 (id 277)
INFO:root:peer nb: 160 (id 306)
INFO:root:peer nb: 170 (id 326)
INFO:root:peer nb: 180 (id 342)
INFO:root:peer nb: 190 (id 356)
INFO:root:peer nb: 200 (id 370)
INFO:root:peer

KeyboardInterrupt: 

In [81]:
%timeit cm_movie = cor_matrix(base='movie', verbose=True)
cm_movie.head()

/Users/anitaetemiliedurr/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
INFO:root:nb of peers: 260
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 24)
INFO:root:peer nb: 30 (id 40)
INFO:root:peer nb: 40 (id 60)
INFO:root:peer nb: 50 (id 79)
INFO:root:peer nb: 60 (id 100)
INFO:root:peer nb: 70 (id 123)
INFO:root:peer nb: 80 (id 152)
INFO:root:peer nb: 90 (id 164)
INFO:root:peer nb: 100 (id 177)
INFO:root:peer nb: 110 (id 202)
INFO:root:peer nb: 120 (id 223)
INFO:root:peer nb: 130 (id 235)
INFO:root:peer nb: 140 (id 260)
INFO:root:peer nb: 150 (id 277)
INFO:root:peer nb: 160 (id 306)
INFO:root:peer nb: 170 (id 326)
INFO:root:peer nb: 180 (id 342)
INFO:root:peer nb: 190 (id 356)
INFO:root:peer nb: 200 (id 370)
INFO:root:peer

KeyboardInterrupt: 

In [358]:
user = 4
movie = 49280
movie2 = 26791

p1 = predict_collabo(user, movie, base='user')
p2 = predict_collabo(user, movie, base='movie')
p3 = predict_collabo(user, movie2, base='user')
p4 = predict_collabo(user, movie2, base='movie')
print(p1)
print(p2)
print(p3)
print(p4)

4.04
3.0
4.04
3.0


# <span style="color:red"> Hypothèse sur pourquoi c'est pas différent (en haut) : pas assez de données. Parce que quand on utilise tout le datase (en bas) ça marche ... ?
</span>.

### Utilisation de toutes les données

In [38]:
dfr = ratings

In [39]:
%timeit normalize(base='user')
dfr.head()

1min 18s ± 2.94 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


,userId,movieId,rating,rating_norm_user
0,1,31,2.5,-0.05
1,1,1029,3.0,0.45
2,1,1061,3.0,0.45
3,1,1129,2.0,-0.55
4,1,1172,4.0,1.45


In [40]:
%timeit normalize(base='movie')
dfr.head()

KeyboardInterrupt: 

In [269]:
t1 = time()
cm_user = cor_matrix(base='user', verbose=True)
t2 = time()
print(t2-t1)
cm_user.head()

INFO:root:nb of peers: 671
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 21)
INFO:root:peer nb: 30 (id 31)
INFO:root:peer nb: 40 (id 41)
INFO:root:peer nb: 50 (id 51)
INFO:root:peer nb: 60 (id 61)
INFO:root:peer nb: 70 (id 71)
INFO:root:peer nb: 80 (id 81)
INFO:root:peer nb: 90 (id 91)
INFO:root:peer nb: 100 (id 101)
INFO:root:peer nb: 110 (id 111)
INFO:root:peer nb: 120 (id 121)
INFO:root:peer nb: 130 (id 131)
INFO:root:peer nb: 140 (id 141)
INFO:root:peer nb: 150 (id 151)
INFO:root:peer nb: 160 (id 161)
INFO:root:peer nb: 170 (id 171)
INFO:root:peer nb: 180 (id 181)
INFO:root:peer nb: 190 (id 191)
INFO:root:peer nb: 200 (id 201)
INFO:root:peer nb: 210 (id 211)
INFO:root:peer nb: 220 (id 221)
INFO:root:peer nb: 230 (id 231)
INFO:root:peer nb: 240 (id 241)
INFO:root:peer nb: 250 (id 251)
INFO:root:peer nb: 260 (id 261)
INFO:root:peer nb: 270 (id 271)
INFO:root:peer nb: 280 (id 281)
INFO:root:peer nb: 290 (id 291)
INFO:root:peer nb: 300 (id 301)
INF

4818.186100959778


correlation
u v              
1 4     -0.515315
  5     -1.000000
  7      0.200515
  9      1.000000
  15     0.043773

In [466]:
t1 = time()
cm_movie = cor_matrix(base='movie', verbose=True)
t2 = time()
print(t2-t1)
cm_movie.head()

INFO:root:nb of peers: 9066
INFO:root:peer nb: 0 (id 1)
INFO:root:peer nb: 10 (id 11)
INFO:root:peer nb: 20 (id 21)
INFO:root:peer nb: 30 (id 31)
INFO:root:peer nb: 40 (id 42)
INFO:root:peer nb: 50 (id 53)
INFO:root:peer nb: 60 (id 64)
INFO:root:peer nb: 70 (id 76)
INFO:root:peer nb: 80 (id 86)
INFO:root:peer nb: 90 (id 98)
INFO:root:peer nb: 100 (id 110)
INFO:root:peer nb: 110 (id 122)
INFO:root:peer nb: 120 (id 137)
INFO:root:peer nb: 130 (id 151)
INFO:root:peer nb: 140 (id 161)
INFO:root:peer nb: 150 (id 171)
INFO:root:peer nb: 160 (id 181)
INFO:root:peer nb: 170 (id 193)
INFO:root:peer nb: 180 (id 204)
INFO:root:peer nb: 190 (id 216)
INFO:root:peer nb: 200 (id 228)
INFO:root:peer nb: 210 (id 238)
INFO:root:peer nb: 220 (id 248)
INFO:root:peer nb: 230 (id 258)
INFO:root:peer nb: 240 (id 268)
INFO:root:peer nb: 250 (id 278)
INFO:root:peer nb: 260 (id 290)
INFO:root:peer nb: 270 (id 302)
INFO:root:peer nb: 280 (id 314)
INFO:root:peer nb: 290 (id 325)
INFO:root:peer nb: 300 (id 335)
IN

KeyboardInterrupt: 

In [461]:
user = 3
movie = 4
movie2 = 5765

p1 = predict_collabo(user, movie, base='user')
# p2 = predict_collabo(k, user, movie, base='movie')
p3 = predict_collabo(user, movie2, base='user')
print(p1)
# print(p2)
print(p3)

3.389937106918239
3.6666666666666665


## Modèle hybride : popularité par genre et collaborative filtering

La méthode précédente permet de prédire une note, mais nous aimerions pouvoir recommander des films à un utilisateurs qu'il est susceptible d'aimer. Pour cela il faudrait prédire la note qu'il donnerait à tous les films qu'il n'a pas encore noté et prélever ceux dont la note prédite est la plus élevée. Ceci serait beaucoup trop coûteux. Une première solution est de ne considérer que des films appartenant à ses genres préférés. Ceci risque d'être toujours trop coûteux, alors nous allons nous restreindre qu'aux films les plus populaires dans ses genres préférés.

# <span style="color:green"> df_filtered ou dfm avec le merge dans return ? </span>

# <span style="color:green"> iterer de manière plus efficace ?</span>

In [348]:
def hybride_genre(uid, nb_reco=10, base='user', k=4):
    '''
    Prédit des notes que l'utilisateur uid donnerait aux 1000 films les plus populaires
    appartenant à ses genres préférés
    Retourne les nb_reco films dont le système prédit la meilleure note
    '''
    dfm_filtered = top_reco(uid, 1000)
    
    predictions = pd.DataFrame(columns=['movieId', 'predict_rating'])
    
    for mid in dfm_filtered.movieId.unique():
        p, o = (uid, mid) if base == 'user' else (mid, uid)
        rat = predict_collabo(p, o, base, k)
        predictions = predictions.append({'movieId':int(mid), 'predict_rating':rat}, ignore_index=True)
    
    predictions = predictions.sort_values(by='predict_rating', ascending=False)
    reco = 
    
    reco = predictions.head(nb_reco) if predictions.shape[0] > nb_reco else predictions
    
    return dfm_filtered.merge(reco, how='inner')

In [345]:
user = 4

hybride_genre(user)

       movieId                       title  mean_rating    genres
44988    49280            The One-Man Band          5.0  Thriller
16885    26791                Brigham City          5.0     Drama
23014     1428  Once Upon a Time in Mexico          5.0    Action
24161     5765                Knight Moves          5.0  Thriller
24182     3112     The Night of the Hunter          5.0  Thriller


,movieId,genres,title,mean_rating,predict_rating
0,2649,"[Drama, Thriller, Mystery]",The Game,5.0,5.0
1,3021,"[Horror, Thriller]",1408,5.0,5.0
2,1819,"[Comedy, Romance]","You, Me and Dupree",5.0,5.0
3,4201,"[Action, History]",The Fifth Musketeer,5.0,5.0
4,8699,[Comedy],Anchorman: The Legend of Ron Burgundy,5.0,5.0
5,4584,"[Drama, Romance]",Sense and Sensibility,5.0,5.0
6,25801,"[Foreign, Comedy, Drama]",Aamdani Atthanni Kharcha Rupaiya,5.0,5.0
7,26791,"[Crime, Drama, Mystery, Thriller]",Brigham City,5.0,5.0
8,4459,[Thriller],Night Without Sleep,5.0,5.0
9,31973,"[Drama, History]",Soldier of God,5.0,5.0


Cette solution présente néanmoins un désavantage. Premièrement, seuls les films les plus populaires sont considérés, leur donnant plus de visibilté parmis les utilisateurs. Ainsi un nouveau film qui n'aura pas beaucoup été vu aura que peut de chance d'être recommandé par assez populaire. Le deuxième problème est que cette méthode regroupe les films par genres. Or ce qui caractérise un film est plus vaste que seulement la case dans laquelle il s'inscrit et peut dépendre du réalisateur, du lieu de tournage ou de pleins d'autres facteurs. C'est ici que le *clustering* nous vient en aide. Cela permet de regrouper les films selon leurs similitudes issues de méta-informations et ainsi d'affiner la recherche.

## Clustering des films

## Model-based recommendation system

La matrice des notes user-item $R$ est partiellement vide. Ainsi réduire les dimensions de la matrice pourrait améliorer la complexité de nos algorithmes. Une méthode que nous pourrions avoir envie d'utiliser est la décomposision en valeurs singulières : $R = U_{svd} \Sigma V_{svd}$. Cependant cette méthode ne s'applique pas ici étant donné que $R$ n'est pas complète et qu'on a besoin de réaliser des calculs algébriques avec $R$ pour trouver la décomposition.

On considère donc un modèle dans lequel il existe des attributs décrivants les films et les préférences des utilisateurs. La matrice $R$ peut alors être factorisée en produit de deux matrices $U$ et $V$ représentant respectivement les utilisateurs et les items :

$$
R \approx U \times V^T
$$

avec $R \in \mathbb{R}^{n \times m}$ la matrice des notes user-item, $U \in \mathbb{R}^{n \times \ell}$ la matrice des users, $V \in \mathbb{R}^{m \times \ell}$ la matrice des items et $\ell$ le nombre d'attributs. Pour faire un rapprochement avec la SVD, on peut considerer que $U = U_{svd} \Sigma^{1/2}$ et $V = \Sigma^{1/2} V_{svd}$. On note $U_i$ les lignes de $U$ et $V_j$ les lignes de $V$ :
$
U = \left[ \begin{array}{c} U_1 \\ \vdots \\ U_n \end{array} \right]
$ et 
$
V = \left[ \begin{array}{c} V_1 \\ \vdots \\ V_m \end{array} \right]
$
avec $U_i^T, V_j^T \in \mathbb{R^\ell}$.

Dans ce modèle, chaque note $R_{ij}$ associée à un couple user-item $(i, j)$ est le résultat du produit scalaire entre la ligne associée au user $i$ dans $U$ et la ligne associée au item $j$ dans $V$ : $R_{ij} = U_i \cdot V_j^T$. Une fois les matrices $U$ et $V$ apprises, pour prédire une note il suffira de faire le produit scalaire entre les lignes associées.

Trouver $U$ et $I$ revient à minimiser l'erreur entre la note prédite $U_i \cdot V_j^T$ et la véritable note $R_{ij}$. Il s'agit du problème de minimisation suivant, avec $E = \{(i, j) \mbox{ | } R_{ij} \mbox{ connue}\}$ :

$$
(U, V) = argmin_{(U, V)} \sum_{(i, j) \in E} [U_i \cdot V_j^T - R_{ij}]^2
$$

qui est équivalent à:

$$
(U, V) = argmin_{(U, V)} \frac{1}{2}\sum_{(i, j) \in E} [U_i \cdot V_j^T - R_{ij}]^2 + \lambda (\|U_i\|^2 + \|V_j\|^2)
$$

Le terme de droite est un terme régulateur, de paramètre $\lambda$ à ajuster, permettant de prévenir un overfitting.

Pour résoudre ce problème, nous allons utiliser une méthode de descente de gradient.


*Pour résoudre ce problème, on peut utiliser une méthode de descente de gradient. Nous allons ensuite optimiser cette méthode en utilisant d'abord des batch, puis en se réduisant à un problème de moindre carré en fixant alternativement les matrices $U$ et $V$.*

### Descente de gradient (à pas constant)

Dans notre [cours d'optimisation](https://www.ceremade.dauphine.fr/~gontier/enseignement.html) donné par David Gontier, nous avons étudié différentes méthodes de descente de gradient de complexité et d'optimalité différentes. Cependant il nous semble qu'utiliser une version simple à pas $\tau$ constant suffit. Il sera possible de régler cet hyper-paramètre par validation croisée. 

Notre fonction objective est la suivante :
$$
F(U, V) := \sum_{(i, j) \in E} \frac{1}{2}[U_i \cdot V_j^T - R_{ij}]^2 + \frac{\lambda}{2} (\|U_i\|^2 + \|V_j\|^2)
$$

Dans une descente de gradient classique, à chaque itération on met à jour $U$ et $V$ suivant la formule 
$
(U, V) = (U, V) - \tau \nabla F(U, V)
$. Cependant, dans notre cas nous n'allons pas mettre à jour toutes les lignes de $U$ et $V$ simultanément. En effet, puisque la somme dans $F$ ne se fait que sur les couples $(i, j)$ pour lesquels la note est connue, nous allons seulement mettre à jour le couple $(U_i, V_j)$ associé en itérant sur tous les couples $(i, j) \in E$. 

Pour une note $R_{ij}$, on a 
$
\frac{\partial F}{\partial U_i} = V_j^T (U_i \cdot V_j^T - R_{ij}) + \lambda U_i
$
 et 
$
\frac{\partial F}{\partial V_j} = Ui (U_i \cdot V_j^T - R_{ij}) + \lambda V_j
$
donc on peut mettre à jour les lignes $U_i$ et $V_j$ selon les formules 
$$
U_i = Ui - \tau [V_j^T (U_i \cdot V_j^T - R_{ij}) + \lambda U_i]\\
V_j = V_j - \tau [Ui (U_i \cdot V_j^T - R_{ij}) + \lambda V_j]
$$

In [51]:
dfr = ratings.copy()
user_rank = dfr.userId.unique().tolist()
movie_rank = dfr.movieId.unique().tolist()
R = [ratings.loc[ratings['userId'] == user, ['rating']].to_numpy() for user in user_rank]

In [387]:
R.head()

,userId,movieId,rating,rating_norm_user
0,1,31,2.5,-0.05
9,1,1343,2.0,-0.55
2,1,1061,3.0,0.45
3,1,1129,2.0,-0.55
4,1,1172,4.0,1.45


In [388]:
n = len(R.userId.unique())
m = len(R.movieId.unique())
print('n:', n, 'm:', m)

n: 671 m: 9066


In [393]:
a = R.movieId.unique()
a.sort()
print(a)
R.userId.unique()[:100]

[     1      2      3 ... 162542 162672 163949]


array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100])

Il se peut que tous les entiers entre 1 et $n$ (ou $m$) ne soient pas utlisés par les id des users (ou des movies). Ceci est par exemple le cas lorsqu'on travaille avec un échantillon des données. Puisque nous aimerions utiliser des numpy array dans nos calculs, il va être nécessaire d'avoir la correspondant entre les id et les indices utilisés dans les numpy array (que nous allons appeler rang). Pour cela, utilisons simplement une liste contenant les id et dont l'indice dans la liste d'un id donné correspondra au rang. Pour trouver l'id à partir d'un rang il suffira de faire un simple extraction, pour trouver le rang à partir d'un id on utilisera la méthode `index()`.

In [394]:
user_rank = R.userId.unique().tolist()
movie_rank = R.movieId.unique().tolist()

La matrice $R$ étant vide, nous n'allons pas utiliser de matrice pour la représenter et garderons la dataframe qui ne contient que les notes connues. Nous allons également avoir besoin d'écrire une fonction `get_rat()` qui permet d'accéder à la note d'un couple de rang dans la dataframe des notes. Nous utilisons également une fonction `known()` pour construire l'ensemble $E$.

# <span style="color:green"> Comparer si get_rat plus efficace avec R une dataframe ou R un numpy matrix </span>

In [395]:
def get_rat(R, i, j):
    '''
    Retourne la note de rang (i, j) dans la dataframe R
    '''
    uid, mid = user_rank[i], movie_rank[j]
    return float(R.loc[(R['userId'] == uid) & (R['movieId'] == mid), 'rating'])

In [396]:
def known(R):
    '''
    Retourne l'ensemble des indices (i, j) pour lesquels la note est connue dans R
    '''
    ids = set(R.loc[:, ['userId', 'movieId']].itertuples(index=False, name=None))
    E = set(map(lambda t : (user_rank.index(t[0]), movie_rank.index(t[1])), ids ))
    return E

Nous pouvons à présent écrire la fonction résolvant notre problème de minimisation. Remarquons qu'elle modifie les valeurs de $U$ et $V$ en place.

# <span style="color:green"> Condition d'arrêt de la descente : prends trop de temps à calculer </span>

In [449]:
def compute_rmse(U, V, E):
    predicted = np.dot(U, V.T) # the predicted rating matrix
    
    rmse = 0
    nb_instances = 0
    for (i, j) in E :
        rmse += (predicted[i, j] - get_rat(R, i, j)) ** 2 
        nb_instances += 1
    return np.sqrt(rmse / nb_instances)

In [453]:
def descenteGradient(U, V, tau, tol=1e-3, Niter=100, verbose=False):
    E = known(R)
    last_rmse = 0
    
    if verbose:
        logger = logging.getLogger()
        logger.setLevel(logging.INFO)
        logging.info('nombre de couples : {}'.format(len(E)))

    for n in range(Niter):
        if verbose :
            logging.info("iteration: {}".format(n))
            logging.info('compute rmse')
        
        rmse = compute_rmse(U, V, E)
        if abs(rmse - last_rmse) < tol:
            return U, V
        last_rmse = rmse

        if verbose:
            logging.info('update U and V')
        
        n_couple = 0
        for (i, j) in E :
            if verbose and not n_couple % 10000:
                logging.info("couple number: {}".format(n_couple))
            n_couple += 1
    
            gradU = V[j].T * (np.dot(U[i], V[j].T) - get_rat(R, i, j)) + lamb * U[i]
            gradV = U[i] * (np.dot(U[i], V[j].T) - get_rat(R, i, j)) + lamb * V[j]
            U[i] = U[i] - tau * gradU
            V[j] = V[j] - tau * gradV
    print("Erreur, l’algorithme n’a pas convergé après", Niter ," itérations")
    return U, V

In [454]:
# hyper-paramètres à tuned
ell = 10
tau = 1/10
lamb = 1/2
U, V = np.random.rand(n, ell), np.random.rand(m, ell)
print(U[:5, :5])
print(V[:5, :5])

[[0.55204517 0.58170198 0.1518093  0.31525872 0.36633203]
 [0.53422442 0.53602512 0.68000442 0.23003917 0.13306995]
 [0.1010032  0.98588448 0.73738854 0.756818   0.2261537 ]
 [0.1115403  0.37201069 0.82455506 0.16504009 0.41905298]
 [0.83801501 0.18104372 0.51859209 0.9051353  0.15224785]]
[[0.83501302 0.90172618 0.89503448 0.40120639 0.18555992]
 [0.27463729 0.81252973 0.17824317 0.24591062 0.63322698]
 [0.54125742 0.58939532 0.52880285 0.30253376 0.05032937]
 [0.55586891 0.73201994 0.74266666 0.37488648 0.24952532]
 [0.15904854 0.98802114 0.55517043 0.58776976 0.84335749]]


In [455]:
t1 = time()
descenteGradient(U, V, tau, verbose=True)
t2 = time()
print(t2-t1)

INFO:root:nombre de couples : 100004
INFO:root:iteration: 0
INFO:root:compute rmse
INFO:root:update U and V
INFO:root:couple number: 0
INFO:root:couple number: 10000
INFO:root:couple number: 20000
INFO:root:couple number: 30000
INFO:root:couple number: 40000
INFO:root:couple number: 50000
INFO:root:couple number: 60000
INFO:root:couple number: 70000
INFO:root:couple number: 80000
INFO:root:couple number: 90000
INFO:root:couple number: 100000
INFO:root:iteration: 1
INFO:root:compute rmse
INFO:root:update U and V
INFO:root:couple number: 0
INFO:root:couple number: 10000
INFO:root:couple number: 20000
INFO:root:couple number: 30000
INFO:root:couple number: 40000
INFO:root:couple number: 50000
INFO:root:couple number: 60000
INFO:root:couple number: 70000
INFO:root:couple number: 80000
INFO:root:couple number: 90000
INFO:root:couple number: 100000
INFO:root:iteration: 2
INFO:root:compute rmse
INFO:root:update U and V
INFO:root:couple number: 0
INFO:root:couple number: 10000
INFO:root:couple

3738.3853256702423


In [456]:
t1 = time()
rmse = compute_rmse(U, V, known(R))
t2 = time()
print(t2-t1)
print(rmse)

KeyboardInterrupt: 

In [457]:
def predict_model(uid, mid):
    user = user_rank[uid]
    movie = movie_rank[mid]
    return np.dot(U[user], V[movie].T)

In [458]:
user = 3
movie = 4

print(predict_model(user, movie))

3.941011674818386


### Cross-validation


## Linear model : content-based

On remarque que si $U$ ou $V$ est fixé, la fonction objective devient quadratique. Or nous connaissons des algorithmes efficaces pour minimiser des fonctions quadratiques. De plus, une matrice d'attributs des films peut être donnée puisqu'on connaît certaines informations sur les films.